In [45]:
import os, sys
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
from preprocess import *

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time, datetime 
import mlflow

dir = '../data/'

train = pd.read_csv(dir+'train.csv')
test = pd.read_csv(dir+'test.csv')
submission = pd.read_csv(dir+'sample_submission.csv')

X, y = set_train_feature_target(train, user_id='user_id', target='target')
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = split_data(X, y)

(10000, 13) (10000,)


In [62]:
num = X.select_dtypes([np.number]).columns
cat = X.select_dtypes(exclude=[np.number]).columns
num, cat

(Index(['subscription_duration', 'recent_login_time', 'average_login_time',
        'average_time_per_learning_session', 'monthly_active_learning_days',
        'total_completed_courses', 'recent_learning_achievement',
        'abandoned_learning_sessions', 'community_engagement_level',
        'customer_inquiry_history', 'payment_pattern'],
       dtype='object'),
 Index(['preferred_difficulty_level', 'subscription_type'], dtype='object'))

In [54]:
x_train_t = np.load('test.npy')
x_train_t[0]

array([-0.13147076, -1.44151953,  0.10110535, -0.34655895, -0.51575377,
        0.75999783, -1.37500813,  0.54822666,  0.88668639, -0.70623307,
        1.07167825,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        0.        ])

In [52]:
import os, sys
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
from dataloader import get_path, read_df
from preprocess import create_preprocessor
from pipeline import create_pipeline
from crossvalidate import CrossValidate
import mlflow
from mlflow_utils import set_or_create_experiment
from models import create_models_pipeline, models
from sklearn.pipeline import Pipeline

#mlflow experiment 환경설정
experiment_name = 'subscription_classification'
set_or_create_experiment(experiment_name=experiment_name)

# train = pd.read_csv(dir+'train.csv')
# test = pd.read_csv(dir+'test.csv')
# X, y = set_train_feature_target(train, user_id='user_id', target='target')
# print(X.shape, y.shape)
# X_train, X_test, y_train, y_test = split_data(X, y)

preprocessor = create_preprocessor(X)
preprocessor
X_train_t = preprocessor.fit_transform(X_train)
X_test_t = preprocessor.transform(X_test)

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. 'Pipeline(steps=[('payment',
                      payment_1 payment_2 payment_3
0            1         0         1
1            1         1         0
2            1         1         1
3            0         0         0
4            0         0         0
...        ...       ...       ...
9995         0         0         1
9996         1         1         1
9997         1         1         0
9998         0         0         0
9999         0         1         1

[10000 rows x 3 columns])])' (type <class 'sklearn.pipeline.Pipeline'>) doesn't.

In [39]:
train.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,b919c29d,13,14,14.946163,8.427187,18,16,68.360455,3,4,Low,Basic,4,5,0
1,a0a60abb,16,18,18.453224,72.646087,16,13,97.567322,2,3,Medium,Basic,1,6,1
2,b9f171ae,22,1,16.195228,21.774492,13,14,94.358763,3,4,Medium,Premium,0,7,1
3,5dc0ba8b,1,19,17.628656,42.659066,19,18,70.153228,0,3,Low,Basic,1,0,1
4,65c83654,4,5,21.390656,30.744287,19,10,81.917908,2,4,Medium,Basic,3,0,1


In [23]:
# X.columns
difficulty = 'preferred_difficulty_level'
sub_type = 'subscription_type'
pattern = 'payment_pattern'

mapping_difficulty = {'Low': 0, 'Medium': 1, 'High': 2}
mapping_sub_type = {'Basic': 0, 'Premium': 1}
mapping_payment = {0 : [0, 0, 0]
                ,1 : [0, 0, 1]
                ,2 : [0, 1, 0]
                ,3 : [0, 1, 1]
                ,4 : [1, 0, 0]
                ,5 : [1, 0, 1]
                ,6 : [1, 1, 0]
                ,7 : [1, 1, 1]
                }
def map_payment(X, payment_col):
    mapping_payment = {0 : [0, 0, 0]
                    ,1 : [0, 0, 1]
                    ,2 : [0, 1, 0]
                    ,3 : [0, 1, 1]
                    ,4 : [1, 0, 0]
                    ,5 : [1, 0, 1]
                    ,6 : [1, 1, 0]
                    ,7 : [1, 1, 1]
                    }

    mapped = X['payment_pattern'].map(mapping_payment).to_list()
    return np.array(mapped)
patt = map_payment(X, pattern)
# pattpipe = make_pipeline(patt)
pattpipe = Pipeline(steps=[('1', map_payment(X, pattern))])
pattpipe

TypeError: [[1 0 1]
 [1 1 0]
 [1 1 1]
 ...
 [1 1 0]
 [0 0 0]
 [0 1 1]] is not an estimator instance.

TypeError: [[1 0 1]
 [1 1 0]
 [1 1 1]
 ...
 [1 1 0]
 [0 0 0]
 [0 1 1]] is not an estimator instance.

Pipeline(steps=[('1',
                 array([[1, 0, 1],
       [1, 1, 0],
       [1, 1, 1],
       ...,
       [1, 1, 0],
       [0, 0, 0],
       [0, 1, 1]]))])

In [18]:
from preprocess import MappingTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
difficulty_transformer = MappingTransformer(mapping=mapping_difficulty)
sub_type_transformer = MappingTransformer(mapping=mapping_sub_type)

diffpipe = make_pipeline(MappingTransformer(mapping=mapping_difficulty))
subpipe = make_pipeline(MappingTransformer(mapping=mapping_sub_type))

diffpipe.fit_transform(X_train[difficulty])
prep = ColumnTransformer(
    transformers=[
        ('diff', diffpipe, difficulty),
        ('sub', subpipe, sub_type),
        ('patt', pattpipe,pattern)
    ]
)
prep.fit_transform(X_train)

TypeError: All estimators should implement fit and transform, or can be 'drop' or 'passthrough' specifiers. 'Pipeline(steps=[('ndarray',
                 array([[1, 0, 1],
       [1, 1, 0],
       [1, 1, 1],
       ...,
       [1, 1, 0],
       [0, 0, 0],
       [0, 1, 1]]))])' (type <class 'sklearn.pipeline.Pipeline'>) doesn't.

In [8]:
mapping_difficulty = {'Low': 0, 'Medium': 1, 'High': 2}
difficulty_transformer = MappingTransformer(mapping=mapping_difficulty)
difficulty_transformer.fit_transform(X['preferred_difficulty_level']).reshape(-1, 1)

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit_transform(X['preferred_difficulty_level'])

array([[0],
       [1],
       [1],
       ...,
       [2],
       [0],
       [0]], dtype=int64)

In [27]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# 매핑 딕셔너리 정의
mapping_difficulty = {'Low': 0, 'Medium': 1, 'High': 2}
mapping_sub_type = {'Basic': 0, 'Premium': 1}
mapping_payment = {
    0: [0, 0, 0],
    1: [0, 0, 1],
    2: [0, 1, 0],
    3: [0, 1, 1],
    4: [1, 0, 0],
    5: [1, 0, 1],
    6: [1, 1, 0],
    7: [1, 1, 1]
}

# 사용자 정의 변환기 생성
class DictMapper(BaseEstimator, TransformerMixin):
    def __init__(self, mapping):
        self.mapping = mapping

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.replace(self.mapping).values.reshape(-1, 1)

# payment 변환 함수 정의
def payment_transformer(X):
    return np.array([mapping_payment[x] for x in X])

# 데이터 예시
data = pd.DataFrame({
    'difficulty': ['Low', 'Medium', 'High'],
    'sub_type': ['Basic', 'Premium', 'Basic'],
    'payment': [0, 3, 7]
})

# ColumnTransformer 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('difficulty', DictMapper(mapping_difficulty), 'difficulty'),
        ('sub_type', DictMapper(mapping_sub_type), 'sub_type'),
        ('payment', FunctionTransformer(payment_transformer, validate=False), 'payment')
    ]
)

# Pipeline 생성
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# 데이터 변환
transformed_data = pipeline.fit_transform(data)

transformed_data


C:\Users\lys45\AppData\Local\Temp\ipykernel_19292\1974760913.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return X.replace(self.mapping).values.reshape(-1, 1)


array([[0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1],
       [2, 0, 1, 1, 1]], dtype=int64)

In [64]:
preprocessor = create_preprocessor1(X)
X_train_t = preprocessor.fit_transform(X_train)
X_test_t = preprocessor.transform(X_test)

ValueError: The output of the 'difficulty' transformer should be 2D (numpy array, scipy sparse array, dataframe).

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time, datetime 
import mlflow

dir = '../data/'

train = pd.read_csv(dir+'train.csv')
test = pd.read_csv(dir+'test.csv')
submission = pd.read_csv(dir+'sample_submission.csv')

train.head()

from sklearn.model_selection import train_test_split

X = train.drop(['user_id', 'target'], axis=1)
y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder

class num_scaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ss = StandardScaler()

    def fit(self, X, y=None):
        self.ss.fit(X)
        return self 
    
    def transform(self, X, y=None):
        return self.ss.transform(X)

class label_scaler(BaseEstimator, TransformerMixin):
    def __init__(self, mapping):
        self.mapping = mapping

    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        return X.map(self.mapping).values.reshape(-1, 1)
    
class ohe_scaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ohe = OneHotEncoder(sparse_output=False)

    def fit(self, X, y=None):
        self.ohe.fit(X.values.reshape(-1, 1))
        return self 
    
    def transform(self, X, y=None):
        return self.ohe.transform(X.values.reshape(-1, 1))
    
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
# 수치형에 왜 실수만 넣었냐?
# 요일 수 스케일링 해야하나? 일단 몰라서 passthrough
num_cols = X_train.select_dtypes('float').columns
label_cols = X_train.select_dtypes(exclude=[np.number]).columns[0]
ohe_cols = X_train.select_dtypes(exclude=[np.number]).columns[1]

mapping_difficulty = {'Low' : 0
					  ,'Medium' : 1
					  ,'High' : 2
					  }

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_scaler(), num_cols),
        ('label', label_scaler(mapping=mapping_difficulty), label_cols),
        ('ohe', ohe_scaler(), ohe_cols)
    ],
    remainder='passthrough'
)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier())
])
end_time = time.time()
execution_time = np.round(end_time - start_time, 4)

with mlflow.start_run(run_name='preprocesstime',
                      experiment_id='439615873627327450'):
    mlflow.log_metric('execution time', execution_time)

NameError: name 'start_time' is not defined